<a href="https://colab.research.google.com/github/arpit-parejiya01/Project/blob/main/Resume_content_extrect_using_NER_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Drive_connect**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Installation**

In [ ]:
!pip install pytesseract
!sudo apt update
!sudo apt install tesseract-ocr
!pip install easyocr
!pip install torch torchvision torchaudio


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [999 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,150 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InReleas

# **Import_Library**

In [ ]:
import pytesseract
from PIL import Image
import spacy
from spacy.training.example import Example
from spacy.training import Example
from pathlib import Path
import random
import ast
import pickle
import re
from tqdm import tqdm
import easyocr
import cv2
import re


# Initialize the EasyOCR reader with the desired language
reader = easyocr.Reader(['en'],gpu=True)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

# **Image_Text_Extrect**

In [ ]:
# # Load the image and perform OCR
# img = Image.open('/content/data-scientist.png')
# extracted_text = pytesseract.image_to_string(img)

# Define the path to the image
image_path = '/content/Data Scientist Resume Example.jpeg'

# Read the image using OpenCV
image = cv2.imread(image_path)

# Perform OCR using EasyOCR
results = reader.readtext(image)

# Function to clean unwanted symbols from amounts
def clean_text(text):
    # Remove any unwanted symbols but keep currency signs, numbers, dots, commas, etc.
    cleaned_text = re.sub(r'[^\d\w\s,.₹$€¥¢£]', '', text)
    return cleaned_text

# Extract and clean the text from the OCR results
extracted_text = ""
for (bbox, text, prob) in results:
    # Cleaning up the text, if necessary
    cleaned_text = clean_text(text)
    extracted_text += cleaned_text + "\n"

# Display the extracted and cleaned text
print("Extracted Text:\n", extracted_text)


Extracted Text:
 EMILY SAAVEDRA
88 Hendford Hill, London B22 OGX, United Kingdom  078 3515 0056
emilysaavedragmailcom
Profile
Education
Highly accurate and experienced Data Scientist
IB Diploma Programme
092010
052012
adept at collecting analyzing and interpreting
The International School Estepona, El Paraiso
large datasets, developing
forecasting
Spain
models and performing data management tasks
Possessing an extensive analytical skills, strong
Graduatedwith Distinction Grade 1  Afexcellent
attentionto detail
significant ability to
equivalent in all
subjects
Wvorkn
team environments_
Emily i5 presently
Looking fora Data Scientist position with a
Skills
forwardmoving company
Languages
Work experience
Spanish
Native
English
Full
SpyBiotech, Inc
092017
022019
French
Limited
Data Scientist London United Kingdom
Computer  Data Analytics Skills
Assisted in scientific research on DNA cloning
and analyzed the results_
Microsoft Office
Collected, studied, and interpreted large
MS SQL Server
da

# **Load_Train_Data**

In [ ]:
# Load training data
TRAIN_DATA = open('/content/drive/MyDrive/NER/doccan.txt', 'r').read()
TRAIN_DATA = ast.literal_eval(TRAIN_DATA)

# **Model_Training**

In [ ]:
from __future__ import unicode_literals, print_function

# Function to trim entity spans
def trim_entity_spans(data: list) -> list:
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end

            while valid_start < len(text) and invalid_span_tokens.match(text[valid_start]):
                valid_start += 1

            while valid_end > valid_start and valid_end <= len(text) and invalid_span_tokens.match(text[valid_end - 1]):
                valid_end -= 1

            valid_end = min(valid_end, len(text))
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

# Example usage
TRAIN_DATA = trim_entity_spans(TRAIN_DATA)

# Load or create model
output_dir = Path("ner/")
model = None
n_iter = 500  # Set number of iterations to 200
learning_rate = 0.001  # Adjusted learning rate

# Load or create model
if model is not None:
    nlp = spacy.load(model)
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')
    print("Created blank 'en' model")

# Ensure 'ner' is added
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

# Add labels to NER
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# Training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        print(f"Epoch {itn+1}")
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example],
                drop=0.5,  # Dropout
                sgd=optimizer,
                losses=losses
            )
        print(losses)

Created blank 'en' model
Epoch 1


  4%|▎         | 6/162 [00:00<00:22,  6.96it/s]/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "jeongwukim@email.com  Jeong Wu Ki m (123) 456-7890..." with entities "[[0, 20, 'E-mail'], [22, 35, 'Name'], [36, 50, 'Ph...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
  6%|▌         | 9/162 [00:01<00:29,  5.27it/s]/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Richard Scott  WEB DEVELOPER     SUMMARY  Self-ind..." with entities "[[0, 13, 'Name'], [15, 28, 'Role'], [355, 1324, 'P...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
 10%|█         | 17/162 [0

{'ner': 23218.5711117114}
Epoch 2


100%|██████████| 162/162 [00:18<00:00,  8.72it/s]


{'ner': 16072.920387228532}
Epoch 3


...

{'ner': 242.78437726920302}
Epoch 499


100%|██████████| 162/162 [00:18<00:00,  8.62it/s]


{'ner': 436.65900337068604}
Epoch 500


100%|██████████| 162/162 [00:19<00:00,  8.29it/s]

{'ner': 322.54971345384234}


# **Save_PKL_Model**

In [ ]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)
pickle.dump(nlp, open( "/content/final_1.pkl", "wb" ))

NameError: name 'output_dir' is not defined

# **Final_Result**

In [ ]:
# Test the model on extracted text
tx = " ".join(extracted_text.split('\n'))

doc = nlp(tx)

for ent in doc.ents:
    print(f'{ent.label_}  =============>   {ent.text}\n---------')


Name  =============>   First Last
---------
Exprience  =============>   EXPERIENCE  Resume Worded, New York, NY  Data Scientist  January 2018 - Present  e Proposed and implemented a new targeting model, generating $1M in wealth management sales.  e Led team of 8 peers to research market entry strategies for a new business line, increasing market penetration in 12 states.  e Worked with Sales, IT and marketing teams to optimize key workflows to identify costs, results and opportunities, streamlining business resources by improving internal system integration.  Growthsi, Boston, MA  Associate Data Scientist  July 2015 -
---------
Exprience  =============>   January 2018  e Implemented demand forecasting models, improving data forecast accuracy by 45%, also won ‘Innovation award’ for the same project.  e Contributed to company blog posts, promoting the company's analytics platform, increasing customer traffic by 75%.  e Provided guidance to members and participated in cross-departmental p

# **PKL_Model**

In [ ]:
import pickle
import spacy

# Load the trained NER model from the saved .pkl file
model_path = "/content/final_1 1.pkl"  # Update this path to where your model is saved
with open(model_path, 'rb') as f:
    nlp = pickle.load(f)

image_path = '/content/Data Scientist Resume Example.jpeg'

# Read the image using OpenCV
image = cv2.imread(image_path)

# Perform OCR using EasyOCR
results = reader.readtext(image)

# Function to clean unwanted symbols from amounts
def clean_text(text):
    # Remove any unwanted symbols but keep currency signs, numbers, dots, commas, etc.
    cleaned_text = re.sub(r'[^\d\w\s,.₹$€¥¢£]', '', text)
    return cleaned_text

# Extract and clean the text from the OCR results
extracted_text = ""
for (bbox, text, prob) in results:
    # Cleaning up the text, if necessary
    cleaned_text = clean_text(text)
    extracted_text += cleaned_text + "\n"

# test_text = " ".join(extracted_text.split('\n'))

# Process the text with the loaded model
doc = nlp(extracted_text)

# Print out the entities recognized by the model
for ent in doc.ents:
    print(f'{ent.label_}  =============>   {ent.text}\n---------')



Role  =============>   Data Analytics
---------
PhoneNo  =============>   9590
---------


In [ ]:
extracted_text

'EMILY SAAVEDRA\n88 Hendford Hill, London B22 OGX, United Kingdom  078 3515 0056\nemilysaavedragmailcom\nProfile\nEducation\nHighly accurate and experienced Data Scientist\nIB Diploma Programme\n092010\n052012\nadept at collecting analyzing and interpreting\nThe International School Estepona, El Paraiso\nlarge datasets, developing\nforecasting\nSpain\nmodels and performing data management tasks\nPossessing an extensive analytical skills, strong\nGraduatedwith Distinction Grade 1  Afexcellent\nattentionto detail\nsignificant ability to\nequivalent in all\nsubjects\nWvorkn\nteam environments_\nEmily i5 presently\nLooking fora Data Scientist position with a\nSkills\nforwardmoving company\nLanguages\nWork experience\nSpanish\nNative\nEnglish\nFull\nSpyBiotech, Inc\n092017\n022019\nFrench\nLimited\nData Scientist London United Kingdom\nComputer  Data Analytics Skills\nAssisted in scientific research on DNA cloning\nand analyzed the results_\nMicrosoft Office\nCollected, studied, and interpr